In [ ]:
from skimage import io
import pickle
import sys
import copy
sys.path.append('/Users/jerrytang/.virtualenvs/cv/lib/python2.7/site-packages')

import helpers
import cluster
import evaluation
import registration
import processing
import correction

## Initialization

#### Design Parameters

In [ ]:
# registration
reference_indices = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
match_probability_threshold = 0.2

# clustering
cluster_cutoff_distance = 0.7
cluster_accept_size = 0.001

# assignment
center_assignment_distance = 100

# correction
max_interpolation_length = 20
frame_occurence_threshold = 70

#### Input Paths

In [ ]:
files = ['/Users/jerrytang/hydra_data/corrected_100.csv']
video_path = '/Users/jerrytang/hydra_data/stk_0001_Substack (1-5000).tif'
assignments_read_path = '/Users/jerrytang/hydra_data/motion_assignments.csv'
# None if the loaded data set is not motion corrected (corrected map is output of ICY motion correction)
motion_correction_map_path = '/Users/jerrytang/hydra_data/map_100.csv'

#### Output Paths

In [ ]:
# res is 2 for 512 images, 1 for 1024
res = 2 
video_save_folder = '/Users/jerrytang/hydra_data/temp_video'
assignments_save_path = '/Users/jerrytang/hydra_data/corrected_assignments.csv'

## Tracking

In [ ]:
import pandas as pd
import numpy as np
full = helpers.load_files(files)

In [ ]:
vid = io.imread(video_path)[0:100]

#### Compute New Motion Corrected Tracks

In [ ]:
# registration[time][spot] = vector representation of that spot
registrations = registration.get_soft_registration(full, reference_indices)

In [ ]:
# flat array of nonzero spot vectors
clustering_subset = cluster.get_subset(registrations, clustering_subset_fraction)

In [ ]:
# cluster_assignments[index] is the cluster that the vector at clustering_subset[index] is assigned to
cluster_assignments = cluster.correlation_hac(clustering_subset, cluster_cutoff_distance)

In [ ]:
# clusters[i] = list of indices in clustering_subset for vectors assigned to cluster i
clusters = cluster.reverse_map(cluster_assignments)

In [ ]:
# centers[i] = vector representing the center for the cluster of neuron i
test_cluster_accept_size = 0.0005
centers = cluster.get_centers(clusters, clustering_subset, test_cluster_accept_size)

In [ ]:
# time_assignments[time] = map from spot index to (neuron index, distance)
time_assignments = cluster.assign_neurons(registrations, centers, center_assignment_distance)

In [ ]:
# neurons[n][time] = location of neuron n at time t
neurons = cluster.process_assignments(time_assignments, len(centers), full)

In [ ]:
# if input neurons were motion corrected use corresponding map to translate back into original space
if motion_correction_map_path != None:
    correction_map = processing.load_correction_map("/Users/jerrytang/0_hydra_data/map_100.csv")
    neurons = processing.reverse_correction(neurons, correction_map)

In [ ]:
corrected_full = processing.reverse_full_correction(full, correction_map)

In [ ]:
for i in range(len(corrected_full)):
    corrected_full[i] = np.asarray(corrected_full[i])

In [ ]:
# get distribution of path lengths
import matplotlib.pyplot as plt
% matplotlib inline
plt.hist(list(map(lambda x : len(x), neurons)))

## Postprocessing

In [ ]:
gap_corrected = correction.gap_interpolation(neurons, max_interpolation_length, corrected_full)

In [ ]:
jump_corrected = correction.jump_interpolation(gap_corrected, max_interpolation_length, corrected_full)

In [ ]:
consistent_tracks = correction.filter_consistent_tracks(jump_corrected, 5.0, 1)

In [ ]:
# get distribution of path lengths
import matplotlib.pyplot as plt
% matplotlib inline
plt.hist(list(map(lambda x : len(x), consistent_tracks)))

In [ ]:
assignments_save_path = '/Users/jerrytang/hydra_data/soft.csv'
helpers.write_neuron_assignments(neurons, assignments_save_path, 1, False)

In [ ]:
tracks = evaluation.track_complete(vid, consistent_frame_occurance_filtered, res)
video_save_folder = '/Users/jerrytang/hydra_data'
evaluation.save_vid(tracks, video_save_folder)